In [5]:
import pandas as pd
import deepl
from tqdm import tqdm
import time

# DeepL API 키 설정
translator = deepl.Translator("YOUR_API_KEY")

# CSV 파일 읽기
df = pd.read_csv('')

def translate_batch(texts, batch_size=10):
    translations = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        try:
            # API 호출 제한을 피하기 위한 딜레이
            time.sleep(0.1)
            # 배치 번역
            results = translator.translate_text(batch, source_lang="EN", target_lang="KO")
            translations.extend([result.text for result in results])
        except Exception as e:
            print(f"Error in batch starting at index {i}")
            print(f"Error message: {str(e)}")
            # 에러 발생시 원본 텍스트 사용
            translations.extend(batch)
    return translations


In [ ]:
# 진행상황 표시하면서 배치 단위로 번역
texts = df['text'].tolist()
batch_size = 10  # 배치 크기 조정 가능
num_batches = (len(texts) + batch_size - 1) // batch_size

translated_texts = []
for i in tqdm(range(0, len(texts), batch_size), total=num_batches, desc="Translating batches"):
   batch = texts[i:i+batch_size]
   translations = translate_batch(batch)
   translated_texts.extend(translations)
   
   # 매 50개 샘플마다 중간 결과 확인 및 저장
   if i % (batch_size * 5) == 0 and i > 0:  # 50개 샘플 = batch_size(10) * 5
       print(f"\nProgress check at batch {i//batch_size}:")
       print("\nOriginal vs Translated examples:")
       for orig, trans in zip(batch[:3], translations[-3:]):
           print(f"\nOriginal: {orig}")
           print(f"Translated: {trans}")
       print("\n" + "="*50)
       
       # 현재까지의 번역 결과를 데이터프레임에 업데이트
       temp_df = df.copy()
       temp_df.loc[:len(translated_texts)-1, 'text'] = translated_texts
       
       # 중간 결과 저장
       temp_df.to_csv('', index=False)
       print(f"\nIntermediate results saved at sample {len(translated_texts)}")

# 최종 데이터프레임 업데이트
df['text'] = translated_texts

# 최종 결과 저장
df.to_csv('', index=False)
print("\nFinal results saved.")

# 최종 결과 확인
print("\nFinal result check (first 5 rows):")
print(df[['text']].head())

In [ ]:
# 번역된 텍스트의 기본 통계
print("\nTranslated text statistics:")
print(f"Total number of translated texts: {len(df)}")
print(f"Average length of translated texts: {df['text'].str.len().mean():.1f} characters")